<hr>
<hr>

<div style="text-align: center;">
    <h1>SISTEMA DE RECOMENDAÇÃO DE FILME</h1>
    <p>- Machine Learning -</p>
</div>

<hr>
<hr>

Dados >> https://www.kaggle.com/code/alyssonbispopereira/recomenda-o-de-filmes-ptbr/data

In [2]:
# importando bibliotecas
import pandas as pd
import numpy as np

In [9]:
# importar arquivo com filmes e visualizar primeiras linhas
filmes = pd.read_csv('movies_metadata.csv', low_memory=False )
filmes.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [5]:
# Importando arquivo de avaliações e avaliando as primeiras linhas
avaliacoes = pd.read_csv('ratings.csv')
avaliacoes.head(3)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523


## Pré-Processamento de Dados

In [11]:
# Filtrando somente as colunas necessários e renomeando nome das variaveis

# Seleciona somente as variaveis que iremos utilizar
filmes = filmes[['id','original_title', 'original_language', 'vote_count']]

# Renomeia as variaveis
filmes.rename(columns= {
    'id' : 'ID_FILME',
    'original_title' : 'TITULO',
    'original_language' : 'LINGUAGEM',
    'vote_count' : 'QT_AVALIACOES'
}, inplace=True)

# Exibe as primeiras Linhas do arquivo tratado
filmes.head()

,ID_FILME,TITULO,LINGUAGEM,QT_AVALIACOES
0,862,Toy Story,en,5415.0
1,8844,Jumanji,en,2413.0
2,15602,Grumpier Old Men,en,92.0
3,31357,Waiting to Exhale,en,34.0
4,11862,Father of the Bride Part II,en,173.0


In [12]:
# Filtrando somente as colunas necessários e renomeando nome das variaveis (AVALIAÇÕES)

# Seleciona somente as variaveis que iremos utilizar
avaliacoes = avaliacoes[['userId', 'movieId', 'rating']]

# Renomeia as variaveis
avaliacoes.rename(columns= {
    'userId' : 'ID_USUARIO',
    'movieId' : 'ID_FILME',
    'rating' : 'AVALIAÇÃO'
}, inplace=True)

# Exibe as primeiras Linhas do arquivo tratado
avaliacoes.head()

,ID_USUARIO,ID_FILME,AVALIAÇÃO
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0


In [14]:
# Tratando valores nulos
filmes.isna().sum()

ID_FILME          0
TITULO            0
LINGUAGEM        11
QT_AVALIACOES     6
dtype: int64

In [19]:
# Como são poucos valores nulos, vamos dropar eles porque não terá impacto
filmes.dropna(inplace=True)
filmes.isna().sum()

ID_FILME         0
TITULO           0
LINGUAGEM        0
QT_AVALIACOES    0
dtype: int64

In [18]:
# Verificando se há valores nulos nas avaliaçãoes
avaliacoes.isna().sum()

ID_USUARIO    0
ID_FILME      0
AVALIAÇÃO     0
dtype: int64

In [20]:
# Quantidade de avaliações por usuario 
avaliacoes['ID_USUARIO'].value_counts()

ID_USUARIO
45811     18276
8659       9279
270123     7638
179792     7515
228291     7410
          ...  
30155         1
9641          1
164717        1
243426        1
234625        1
Name: count, Length: 270896, dtype: int64

In [25]:
# Vamos usar o ID_USUARIO que fizeram mais de 999 avaliações (Veremos os usuarios que avaliaram varios filmes, pois esses são fieis nas avaliações)
qt_avaliacoes = avaliacoes['ID_USUARIO'].value_counts() > 999
y = qt_avaliacoes[qt_avaliacoes].index
y.shape

(2509,)